In [1]:
import numpy as np
import pandas as pd

#plot
import matplotlib.pyplot as plt
import seaborn as sns

# dados
import kagglehub
from pathlib import Path
import zipfile


c:\Users\ander\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path = kagglehub.dataset_download("vinayakshanawad/heart-rate-prediction-to-monitor-stress-level")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\ander\.cache\kagglehub\datasets\vinayakshanawad\heart-rate-prediction-to-monitor-stress-level\versions\1


In [4]:
p = Path(path)

train_path = p / "Train Data" / "Train Data Zip"

train_files = list(train_path.rglob("*.csv"))

train_nonL = [f for f in train_files if "non_linear" in f.name.lower()][0]
df_train_nonL = pd.read_csv(train_nonL)
train_time = [f for f in train_files if "time_domain" in f.name.lower()][0]
df_train_time = pd.read_csv(train_time)
train_freq = [f for f in train_files if "frequency_domain" in f.name.lower()][0]
df_train_freq = pd.read_csv(train_freq)
df_train_p1 = df_train_time.merge(df_train_freq, on='uuid', how='inner',validate='one_to_one')
df_train = df_train_p1.merge(df_train_nonL, on='uuid', how='inner',validate='one_to_one')


In [11]:
df = df_train.copy()

1. Verificando presença de ausêntes (ja feita no eda)

In [12]:
df.isna().sum()

MEAN_RR              0
MEDIAN_RR            0
SDRR                 0
RMSSD                0
SDSD                 0
SDRR_RMSSD           0
HR                   0
pNN25                0
pNN50                0
KURT                 0
SKEW                 0
MEAN_REL_RR          0
MEDIAN_REL_RR        0
SDRR_REL_RR          0
RMSSD_REL_RR         0
SDSD_REL_RR          0
SDRR_RMSSD_REL_RR    0
KURT_REL_RR          0
SKEW_REL_RR          0
uuid                 0
VLF                  0
VLF_PCT              0
LF                   0
LF_PCT               0
LF_NU                0
HF                   0
HF_PCT               0
HF_NU                0
TP                   0
LF_HF                0
HF_LF                0
SD1                  0
SD2                  0
sampen               0
higuci               0
datasetId            0
condition            0
dtype: int64

In [13]:
# removendo o que não trará variância
df.drop(['datasetId', 'uuid'], axis=1,inplace=True)
df.head()


,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,KURT,...,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,condition
0,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,-0.856554,...,0.421047,1.514737,3686.666157,65.018055,0.015380,11.001565,199.061782,2.139754,1.163485,no stress
1,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,-0.408190,...,0.070133,0.304603,3006.487251,327.296635,0.003055,9.170129,114.634458,2.174499,1.084711,interruption
2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,0.351789,...,0.512671,1.049528,2685.879461,94.280910,0.010607,11.533417,118.939253,2.135350,1.176315,interruption
3,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,-0.504947,...,0.529387,1.775294,3434.520980,55.328701,0.018074,11.119476,127.318597,2.178341,1.179688,no stress
4,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,-0.548408,...,1.839473,3.279993,2621.175204,29.487873,0.033912,13.590641,87.718281,2.221121,1.249612,no stress


In [14]:
# removendo condition -> classificação e HR -> regressão
df_M = df.drop(['HR', 'condition'], axis=1)
df_M.head(5)

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,...,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci
0,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,11.133333,0.533333,-0.856554,0.335218,...,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,11.001565,199.061782,2.139754,1.163485
1,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,5.600000,0.000000,-0.408190,-0.155286,...,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,9.170129,114.634458,2.174499,1.084711
2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,13.066667,0.200000,0.351789,-0.656813,...,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,11.533417,118.939253,2.135350,1.176315
3,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,11.800000,0.133333,-0.504947,-0.386138,...,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,11.119476,127.318597,2.178341,1.179688
4,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,20.200000,0.200000,-0.548408,-0.154252,...,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,13.590641,87.718281,2.221121,1.249612


2. Lidando com variáveis altamente correlacionadas

In [43]:
threshold = 0.8
preditores = [i for i in df_M.columns]
removidos = []

In [44]:
corr = df_M[preditores].corr(numeric_only=True).abs()
#print(corr)

In [45]:
while True:
    np.fill_diagonal(corr.values, 0) # zera as diagonais
    trigSup = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool)) # trig superior

    # verificando se tem correlação maior que o limiar
    if trigSup.max().max() <= threshold:
        break

    # par mais correlacionado
    A, B = trigSup.stack().idxmax()

    # calculo da média de correlações, removendo com ele prórpio e com a outra escolhida
    mA = corr.loc[A, preditores].drop([A, B]).mean()
    mB = corr.loc[B, preditores].drop([A, B]).mean()

    # indicando remover o preditor com a maior média
    drop = A if mA >= mB else B

    # salvando a mudança
    valor_corr = corr.loc[A, B]
    removidos.append((drop, (A, B), float(valor_corr)))

    # removendo
    preditores.remove(drop)
    corr = corr.drop(index=drop, columns=drop)
    

In [46]:
print(removidos)

[('SD1', ('SDSD', 'SD1'), 1.0000000000000102), ('LF_NU', ('LF_NU', 'HF_NU'), 1.000000000000001), ('KURT', ('KURT', 'KURT_REL_RR'), 1.0), ('SKEW', ('SKEW', 'SKEW_REL_RR'), 1.0), ('RMSSD_REL_RR', ('RMSSD_REL_RR', 'SDSD_REL_RR'), 0.9999999999572793), ('RMSSD', ('RMSSD', 'SDSD'), 0.9999998264707147), ('SDRR', ('SDRR', 'SD2'), 0.9999972215849535), ('HF_NU', ('HF_NU', 'HF_LF'), 0.9961753424035372), ('VLF_PCT', ('VLF_PCT', 'LF_PCT'), 0.9952395745509537), ('MEAN_RR', ('MEAN_RR', 'MEDIAN_RR'), 0.9609491736076482), ('TP', ('VLF', 'TP'), 0.9517997470476465), ('SDSD', ('SDSD', 'pNN25'), 0.9517711780340457), ('pNN25', ('pNN25', 'LF'), 0.9157888760316598), ('SD2', ('SDRR_RMSSD', 'SD2'), 0.9154748170303951), ('SDSD_REL_RR', ('SDRR_REL_RR', 'SDSD_REL_RR'), 0.8994602585830078), ('HF_PCT', ('HF_PCT', 'HF_LF'), 0.8364211887125808)]


In [49]:
df_M

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,...,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci
0,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,11.133333,0.533333,-0.856554,0.335218,...,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,11.001565,199.061782,2.139754,1.163485
1,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,5.600000,0.000000,-0.408190,-0.155286,...,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,9.170129,114.634458,2.174499,1.084711
2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,13.066667,0.200000,0.351789,-0.656813,...,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,11.533417,118.939253,2.135350,1.176315
3,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,11.800000,0.133333,-0.504947,-0.386138,...,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,11.119476,127.318597,2.178341,1.179688
4,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,20.200000,0.200000,-0.548408,-0.154252,...,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,13.590641,87.718281,2.221121,1.249612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369284,721.396910,721.533965,36.377559,9.609941,9.609936,3.785409,0.933333,0.000000,1.007321,0.075965,...,38.227175,5.886975,10.880317,649.351702,8.190908,0.122087,6.797519,50.994582,2.096850,1.314910
369285,984.266492,978.622945,74.918433,13.762274,13.761705,5.443754,6.333333,0.200000,0.248964,0.167716,...,2.573834,0.112964,0.357547,2278.456746,278.683532,0.003588,9.734243,105.502548,2.131264,1.175975
369286,1025.499743,1024.968400,95.309200,23.051254,23.050395,4.134664,30.400000,2.066667,-0.429686,-0.075321,...,4.287216,0.098983,0.180070,4331.259564,554.340761,0.001804,16.304530,133.797793,2.215643,1.140232
369287,798.123167,803.559610,78.449897,16.736686,16.736657,4.687302,10.000000,1.466667,0.600389,-0.634762,...,46.636158,1.591647,3.522995,2930.056129,27.384942,0.036516,11.838553,110.311474,2.091628,1.181937


In [52]:
for a,b,c in removidos:
    df_M.drop([a], axis = 1, inplace=True)

In [53]:
df_M.columns

Index(['MEDIAN_RR', 'SDRR_RMSSD', 'pNN50', 'MEAN_REL_RR', 'MEDIAN_REL_RR',
       'SDRR_REL_RR', 'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR', 'VLF',
       'LF', 'LF_PCT', 'HF', 'LF_HF', 'HF_LF', 'sampen', 'higuci'],
      dtype='object')

In [54]:
from sklearn.preprocessing import StandardScaler

In [57]:
# padronizando e escalonando
scaler = StandardScaler(with_mean=True, with_std=True)
Z = scaler.fit_transform(df_M)
print(Z.mean(), Z.std())

3.492324066073176e-17 1.0


In [59]:
print(Z)

[[ 0.08916087  0.32399012 -0.33596454 ... -0.67275865  0.37334785
  -0.30240555]
 [ 0.80404558 -0.21855867 -0.87458304 ... -0.92307306  0.54119807
  -1.56903886]
 [ 0.49153993 -0.43049566 -0.6726011  ... -0.76971081  0.35207093
  -0.09611535]
 ...
 [ 1.38302122 -0.63414497  1.21256363 ... -0.94848808  0.73996194
  -0.67629604]
 [-0.29025121 -0.52670801  0.60661782 ... -0.24349624  0.1408559
  -0.00571249]
 [-0.20244055 -0.8006104  -0.33596454 ... -0.26188235  0.70397363
   0.83281311]]


In [60]:
Cov = pd.DataFrame(Z, columns=df_M.columns).cov()
print(Cov)

                   MEDIAN_RR  SDRR_RMSSD     pNN50  MEAN_REL_RR  \
MEDIAN_RR           1.000003    0.203816  0.196993    -0.018019   
SDRR_RMSSD          0.203816    1.000003  0.187629     0.014104   
pNN50               0.196993    0.187629  1.000003    -0.039165   
MEAN_REL_RR        -0.018019    0.014104 -0.039165     1.000003   
MEDIAN_REL_RR      -0.006349    0.300670 -0.190604     0.134923   
SDRR_REL_RR        -0.157427   -0.048227  0.756255    -0.013210   
SDRR_RMSSD_REL_RR   0.635820    0.078020  0.012339    -0.035138   
KURT_REL_RR        -0.284508   -0.029829 -0.064264     0.045365   
SKEW_REL_RR        -0.317131    0.313449  0.088570     0.010727   
VLF                 0.399024    0.617611  0.529804    -0.026638   
LF                  0.084147   -0.276739  0.670074    -0.022014   
LF_PCT             -0.262427   -0.637904  0.003415    -0.003950   
HF                 -0.550245   -0.062388  0.412258     0.018999   
LF_HF               0.421339   -0.040054  0.065119     0.00150

In [64]:
vals, vecs = np.linalg.eigh(Cov)              # C é simétrica → eigh
order = np.argsort(vals)[::-1]              # ordem decrescente
vals = vals[order]
vecs = vecs[:, order]
#print(vals, vecs, sep="\n\n")

In [65]:
(vals[0]+vals[1])/vals.sum() * 100

np.float64(46.20636773715332)